# About this project.


Spam messages are unsolicited and unwanted messages. 
Fraudstars use spam messages to trick people into giving them your personal information — things like your password, account number, or Even credit card information.

These messages are designed such a way people fall for it. This is  because it is difficult for people with little knowledge about scams to determine if an sms is from a scammer.


In this project, I will build an application that can help determine if an SMS is spam or not.
The project is all about teaching the computer how to classify messages. 
To do that, i will use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans that understands what spam messages are.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
sms=pd.read_csv("SMSSpamCollection",sep='\t',header=None,names=['Label', 'SMS'])

In [5]:
sms["SMS"]

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: SMS, Length: 5572, dtype: object